### 10.1 Making a Hierarchical Package of Modules

In [ ]:
graphics/
    __init__.py
    primitive/
         __init__.py
         line.py
         fill.py
         text.py
    formats/
         __init__.py
         png.py
         jpg.py

In [ ]:
import graphics.primitive.line
from graphics.primitive import line
import graphics.formats.jpg as jpg

### 10.2 Controlling the Import of Everything

In [2]:
#somemodule.py

def spam():
    pass

def grok():
    pass

blah = 42

__all__ = ['spam', 'grok']

### 10.3 Importing Package Submodules Using Relative names

In [ ]:
mypackage/
    __init__.py
    A/
        __init__.py
        spam.py
        grok.py
    B/
        __init__.py
        bar.py
        
#mypackage/A/spam.py

from . import grok
from ..B import bar

### 10.4 Splitting a Module into Multiple Files

In [ ]:
#my module.py

class A:
    def spam(self):
        print('A.spam')
        
class B(A):
    def bar(self):
        print('B.bar')
        
# __init__.py
from .a import A
from .b import B

### 10.5 Making Separate Directories of Code Import Under a Common Namespace

In [ ]:
foo-package/
    spam/
        blah.py
        
bar-package/
    spam/
        grok.py        

In [ ]:
import sys
sys.path.extend(['foo-package', 'bar-package'])

import spam.blah
import spam.grok

### 10.6 Reloading Modules

In [ ]:
import spam
import imp

imp.reload(spam)

### 10.7. Making a Directory or Zip File Runnable As a Main Script

In [ ]:
myapplication/
    spam.py
    bar.py
    grok.py
    __main__.py

### 10.8. Reading Datafiles Within a Package

In [ ]:
mypackage/
    __init__.py
    somedata.dat
    spam.py
    
#spam.py

import pkgutil
data = pkgutil.gey_data(__package__, 'somedata.dat')

### 10.9 Adding Directories to sys.path

### 10.10 Importing Modules Using a Name Given in a String

In [5]:
import importlib

math = importlib.import_module('math')

math.sin(2)

0.9092974268256817

In [ ]:
mod = importlib.import_module('urllib.request')

u = mod.urlopen('http://www.python.org')

In [ ]:
import importlib

b = importlib.import_module('.b', __package__)

### 10.11. Loading Modules from a Remote Machine Using Import Hooks

### 10.12 Patching Modules on Import

In [9]:
import importlib
import sys
from collections import defaultdict

__post_import_hooks = defaultdict(list)

class PostImportFinder:
    def __init__(self):
        self._skip = set()

    def find_module(self, fullname, path=None):
        if fullname in self._skip:
            return None
        self._skip.add(fullname)
        return PostImportLoader(self)
        
class PostImportLoader:
    def __init__(self, finder):
        self._finder = finder
        
    def load_module(self, fullname):
        importlib.import_module(fullname)
        module = sys.modules[fullname]
        for func in _post_import_hooks[fullname]:
            func(module)
        self._finder._skip.remove(fullname)
        return module

def when_imported(fullname):
    def decorate(func):
        if fullname in sys.modules:
            func(sys.modules[fullname])
        else:
            _post_import_hooks[fullname].append(func)
        return func
    return decorate

In [10]:
sys.meta_path.insert(0, PostImportFinder())

In [ ]:
from postimport import when_imported
@when_imported('threading')

def warn_threads(mod):
    print('Threads? Are you crazy?')

In [ ]:
import threading

In [ ]:
from functools import wraps
from postimport import when_imported

In [ ]:
def logged(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print('Calling', func.__name__, args, kwargs)
        return func(*args, **kwargs)
    return wrapper

# Example
@when_imported('math')
def add_logging(mod):
    mod.cos = logged(mod.cos)
    mod.sin = logged(mod.sin)